In [1]:
from ultralytics import YOLO

In [2]:
import tensorrt as trt

In [3]:
import torch
import torch.nn as nn

In [4]:
from torch.quantization import quantize_dynamic

In [5]:
# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(f"Using device: {device}")

Using device: cuda


In [8]:
model = YOLO("yolov8s_quantized.pt")
model.export(
    format="engine",
    dynamic=True,  
    batch=8,  
    workspace=4,  
    int8=True,
    data="coco.yaml",  
)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.91 🚀 Python-3.8.10 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11009MiB)
YOLOv8s summary (fused): 168 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs

PyTorch: starting from 'yolov8s_quantized.pt' with input shape (8, 3, 640, 640) BCHW and output shape(s) (8, 84, 8400) (21.5 MB)

ONNX: starting export with onnx 1.16.2 opset 17...
ONNX: export success ✅ 3.6s, saved as 'yolov8s_quantized.onnx' (42.6 MB)

TensorRT: starting export with TensorRT 8.6.1...
[09/11/2024-09:32:27] [TRT] [I] The logger passed into createInferBuilder differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[09/11/2024-09:32:27] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 1727, GPU 1326 (MiB)
[09/11/2024-09:32:27] [TRT] [I] ---------------------------------------------

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 5.67MB/s]
Scanning /app/yolospar/datasets/val2017/labels... 5000 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:01<00:00, 3009.87it/s]


New cache created: /app/yolospar/datasets/val2017/labels.cache
[09/11/2024-09:32:32] [TRT] [I] BuilderFlag::kTF32 is set but hardware does not support TF32. Disabling TF32.
[09/11/2024-09:32:32] [TRT] [I] Graph optimization time: 0.00540697 seconds.
[09/11/2024-09:32:32] [TRT] [W] BuilderFlag::kENABLE_TACTIC_HEURISTIC has been ignored in this builder run. This feature is only supported on Ampere and beyond.
[09/11/2024-09:32:32] [TRT] [I] Timing cache disabled. Turning it on will improve builder speed.
[09/11/2024-09:32:34] [TRT] [I] Detected 1 inputs and 3 output network tensors.
[09/11/2024-09:32:34] [TRT] [I] Total Host Persistent Memory: 365728
[09/11/2024-09:32:34] [TRT] [I] Total Device Persistent Memory: 2508800
[09/11/2024-09:32:34] [TRT] [I] Total Scratch Memory: 0
[09/11/2024-09:32:34] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 4 MiB, GPU 300 MiB
[09/11/2024-09:32:34] [TRT] [I] [BlockAssignment] Started assigning block shifts. This will 

'yolov8s_quantized.engine'

In [9]:
# Load the exported TensorRT INT8 model
model = YOLO("yolov8s_quantized.engine", task="detect")

In [10]:
# Run inference
result = model.predict("bus.jpg")

Loading yolov8s_quantized.engine for TensorRT inference...
[09/11/2024-09:48:56] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[09/11/2024-09:48:56] [TRT] [I] Loaded engine size: 13 MiB
[09/11/2024-09:48:56] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +40, now: CPU 0, GPU 40 (MiB)
[09/11/2024-09:48:56] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +1597, now: CPU 0, GPU 1637 (MiB)

image 1/1 /app/convert_to_trt/bus.jpg: 640x640 3 persons, 1 bus, 2.3ms
Speed: 3.4ms preprocess, 2.3ms inference, 295.9ms postprocess per image at shape (1, 3, 640, 640)
